### Language Models

We have to download specific [language models to work with spacy itself](https://spacy.io/usage/models). Each language model contains statistical properties and attributes about the specific language. For example, a common set of language models are 

* `en_core_web_sm`
* `en_core_web_md`
* `en_core_web_lg`

These are the names of the English web (there are other genres) corpus of varying sizes (small, medium, and large).

You can install a specific language model with the following command:

```shell
pip install spacy
python -m spacy download en_core_web_sm
```

### Visualizing Documents
You can use spacy's `displacy` tool to visualize the relationships and meta-information gathered by spacy while ingesting your documents.

In [9]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("I love Professor Chen's class - it is great.")
displacy.render(doc)

### The Spacy Pipeline

When you call `nlp(raw_text)`, spacy performs a series of operations, in sequence, hence a pipeline:

* `Tokenizer`: tokenizes your text to map a raw string document into a series tokens that comprise a `Doc`.
* `Tagger`: tags each of the tokens in a `Doc` with part of speech.
* `Dependency Parser`: parses the tokens to assign dependency relationships (ie. possessive, compound noun, etc.)
* `Named Entity Recognizer`: identifies and labels tokens as named entities.

#### Tokenizing Text

You can quickly retrieve information about your tokens by simply iterating through them using a regular for loop.

In [41]:
text = """Chesa Boudin, the district attorney of San Francisco, 
will face a recall election next June after a backlash in one of 
America's most liberal cities to his policies aimed at reducing 
one hundred thousand people in jails and prisons.
"""

spacy_doc = nlp(text) # spacy_doc is an iterable object of tokens

In [46]:
for token in spacy_doc:
    print(f"""
token: {token.text}, tag:{token.tag_}, lemma:{token.lemma_}, 
entity type: {token.ent_type_}, stopword: {token.is_stop}, like num: {token.like_num}
{spacy.explain(token.tag_)}
    """)


token: Chesa, tag:NNP, lemma:Chesa, 
entity type: PERSON, stopword: False, like num: False
noun, proper singular
    

token: Boudin, tag:NNP, lemma:Boudin, 
entity type: PERSON, stopword: False, like num: False
noun, proper singular
    

token: ,, tag:,, lemma:,, 
entity type: , stopword: False, like num: False
punctuation mark, comma
    

token: the, tag:DT, lemma:the, 
entity type: , stopword: True, like num: False
determiner
    

token: district, tag:NN, lemma:district, 
entity type: , stopword: False, like num: False
noun, singular or mass
    

token: attorney, tag:NN, lemma:attorney, 
entity type: , stopword: False, like num: False
noun, singular or mass
    

token: of, tag:IN, lemma:of, 
entity type: , stopword: True, like num: False
conjunction, subordinating or preposition
    

token: San, tag:NNP, lemma:San, 
entity type: GPE, stopword: False, like num: False
noun, proper singular
    

token: Francisco, tag:NNP, lemma:Francisco, 
entity type: GPE, stopword: False, lik

You can use the [following link](https://spacy.io/usage/linguistic-features#pos-tagging) to reference what each part of speech tag means.

#### Accessing Named Entities
You can quickly identify the named entities in your text using `spacy_doc.ents`. These named entities are identified using a multi-class classifier deep learning model, usually some derivative of a sequence-based neural network, such as a LSTM.

In [51]:
for token in spacy_doc:
    if token.ent_type_:
        print(token.text, token.ent_type_, spacy.explain(token.ent_type_))

Chesa PERSON People, including fictional
Boudin PERSON People, including fictional
San GPE Countries, cities, states
Francisco GPE Countries, cities, states
next DATE Absolute or relative dates or periods
June DATE Absolute or relative dates or periods
one CARDINAL Numerals that do not fall under another type
America GPE Countries, cities, states
one CARDINAL Numerals that do not fall under another type
hundred CARDINAL Numerals that do not fall under another type
thousand CARDINAL Numerals that do not fall under another type


You can use this to quickly count references to certain concepts, topics, or people - in the
below code snippet, we read in 250 BBC Sports football articles, run them through spacy's pipeline, and identify the most recurring entities (`Everton`).

In [95]:
from collections import Counter
documents = []
articles = [f"bbcsport/football/{str(i).zfill(3)}.txt" for i in range(1,250)]

count = Counter()
for article in articles:
    documents.append(nlp(open(article, encoding="latin1").read())) # open each sports article
    count += Counter([entity.text for entity in doc.ents])
count.most_common(5)

[('Everton', 1245),
 ('Gravesen', 996),
 ('the summer', 498),
 ('Real Madrid', 249),
 ('Thomas Gravesen', 249)]

#### Fusing Tokens Together
In the previous example, you can see that `San` and `Francisco` are defined as two separate tokens. Perhaps we want to fuse these tokens together.

In [107]:
with spacy_doc.retokenize() as rtk:
    rtk.merge(spacy_doc[7:9])
for token in spacy_doc[:8]:
    print(token)

Chesa
Boudin
,
the
district
attorney
of
San Francisco, 



#### Get the Document Vector (Average of Each Token's Vector)

In [140]:
import numpy as np
initial_vector = np.zeros(96)

for token in spacy_doc:
    initial_vector += token.vector
initial_vector /= len(spacy_doc)
print(spacy_doc.vector[:5])
print(initial_vector[:5])

[ 0.76455724  0.15433793  0.13041371 -0.3010922   0.00486153]
[ 0.76455719  0.15433793  0.13041371 -0.30109221  0.00486154]


### Using PhraseMatcher

Often, we want to match certain phrases together. We can construct patterns using the 

In [ ]:

from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern = [{"ENT_TYPE": "PERSON", "OP": "+"}, {"POS": "VERB"}]
matcher.add("FULL_NAME", [pattern])
doc = nlp("Elon Musk spends more money than Jeff Bezos. But also Bill burns cash too.")
matched_names = matcher(doc)
for _, start, end in matched_names:
    print(doc[start:end])